<a href="https://colab.research.google.com/github/cdpierse/Siamese-Network/blob/master/siamese_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the entire repo.
!git clone -l -s https://github.com/cdpierse/Siamese-Network.git siamese_network
%cd siamese_network
!ls

Cloning into 'siamese_network'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 6188 (delta 6), reused 9 (delta 3), pack-reused 6173
Receiving objects: 100% (6188/6188), 198.46 MiB | 39.61 MiB/s, done.
Resolving deltas: 100% (35/35), done.
Checking out files: 100% (5957/5957), done.
/content/siamese_network
data		 __init__.py   model.py		      test.py
dockerfile	 loader.py     requirements.txt
environment.yml  makepairs.py  siamese_network.ipynb


In [0]:
#!pip install -r requirements.txt

In [0]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import (Conv2D, Dense, Flatten, Lambda,
                                     MaxPooling2D, Dropout, SpatialDropout2D)
from tensorflow.keras.regularizers import l2
from keras.backend import abs, sum, square, sqrt
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from model import fetch_train_test


In [0]:
def euclidean_distance(left_input,right_input):
  return sqrt(sum(square(left_input-right_input),axis=1,keepdims=True))


In [0]:
def siamese_model(input_shape):
    """Defines architecture for siamese network. Should be two identically
    weighted conv nets with a distance function layer at the end to differentiate 
    and produce a similarity score. This is similarity based metric not a classification 
    task.
    """
    POOL_SIZE = (2, 2)
    KERNEL_SIZE = (3, 3)
    INPUT_SHAPE = input_shape  # not correct

    # declare tensors for two input images to be compared
    left_input = Input(INPUT_SHAPE)
    right_input = Input(INPUT_SHAPE)

    print(left_input.shape)

    # for filters early layers should have less (more abstract) while later layers gain more filters
    model = Sequential()

    model.add(Conv2D(6, KERNEL_SIZE, activation='relu',
                     input_shape=INPUT_SHAPE, kernel_regularizer=l2(0.002)))
    model.add(MaxPooling2D(pool_size=POOL_SIZE))

    model.add(Conv2D(12, KERNEL_SIZE, activation='relu',
                     kernel_regularizer=l2(0.002)))
    model.add(MaxPooling2D(pool_size=POOL_SIZE))

    model.add(Flatten())
    model.add(Dropout(0.45))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.005)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.005)))


    left_image_encoded = model(left_input)
    right_image_encoded = model(right_input)

    contrastive_layer = Lambda(lambda tensors: euclidean_distance(tensors[0],tensors[1]))

    distance = contrastive_layer([left_image_encoded, right_image_encoded])

    prediction = Dense(1, activation='sigmoid')(distance)

    return Model(inputs=[left_input, right_input], outputs=prediction)


In [0]:

def fit_model(lr=None, batch_size=None, val_split=None, epochs=None):
    if lr is None:
        optimizer = tf.keras.optimizers.Adam(0.001)
    else:
        optimizer = tf.keras.optimizers.Adam(lr)
    if batch_size is None:
        batch_size = 32

    if val_split is None:
        val_split = 0.15

    if epochs is None:
        epochs = 100

    train_test_data = fetch_train_test()
    x_train, y_train = train_test_data['x_train'], train_test_data['y_train']
    input_shape = x_train[0][0].shape
    model = siamese_model(input_shape)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    model.fit([x_train[:, 0], x_train[:, 1]], y_train,
              epochs=epochs, batch_size=batch_size, validation_split=val_split)

    x_test, y_test = train_test_data['x_test'], train_test_data['y_test']

    print(model.evaluate([x_test[:, 0], x_test[:, 1]], y_test))
    print(model.metrics_names)
    model.save('model.h5')

In [90]:
fit_model(epochs=300,lr=0.001)

(?, 64, 64, 1)
Train on 6683 samples, validate on 1180 samples
Epoch 1/300
6683/6683 [==============================] - 5s 776us/sample - loss: 1.0441 - acc: 0.5179 - val_loss: 0.7490 - val_acc: 0.5059
Epoch 2/300
6683/6683 [==============================] - 3s 457us/sample - loss: 0.7145 - acc: 0.5460 - val_loss: 0.7098 - val_acc: 0.5339
Epoch 3/300
6683/6683 [==============================] - 3s 447us/sample - loss: 0.6943 - acc: 0.5698 - val_loss: 0.7073 - val_acc: 0.5754
Epoch 4/300
6683/6683 [==============================] - 3s 472us/sample - loss: 0.6884 - acc: 0.5857 - val_loss: 0.6868 - val_acc: 0.6008
Epoch 5/300
6683/6683 [==============================] - 3s 489us/sample - loss: 0.6829 - acc: 0.5975 - val_loss: 0.6988 - val_acc: 0.5720
Epoch 6/300
6683/6683 [==============================] - 3s 498us/sample - loss: 0.6788 - acc: 0.6030 - val_loss: 0.6904 - val_acc: 0.5729
Epoch 7/300
6683/6683 [==============================] - 3s 480us/sample - loss: 0.6785 - acc: 0.6105 -

KeyboardInterrupt: ignored